# Content base recommendation

Import the book profile and user profile and make recommendation

In [37]:
import json

recommendation_user_id = "14796360"
item_profile = {}
file_name = "item_profile_test.txt"
with open(file_name, 'r', encoding='utf-8') as file:
  while (line := file.readline().rstrip()):
    item_profile_data = json.loads(line)
    item_profile_id = list(item_profile_data.keys())[0]
    # Finds the user we want to give recommendation to
    if item_profile_id == recommendation_user_id:
      item_profile = item_profile_data[item_profile_id]
      # print(item_profile.keys())

if item_profile == {}:
  print(f"User id {recommendation_user_id} not found. Please try with another user.")
else:
  print(item_profile.keys())
  print(item_profile['setting'])

dict_keys(['title', 'series', 'author', 'genres', 'pages', 'publisher', 'firstPublishDate', 'awards', 'setting', 'words'])
['District 12, Panem', 'Capitol, Panem', 'Panem (United States)']


In [232]:
import json

#ID od the user we want recommendation for
recommendation_user_id = "62853"

# Finds the user we want to give recommendation to
user_profile = {}
file_name = "user_profiles_TEST.json"
with open(file_name, 'r', encoding='utf-8') as file:
  while (line := file.readline().rstrip()):
    user_profile_data = json.loads(line)
    user_profile_id = list(user_profile_data.keys())[0]
    if user_profile_id == recommendation_user_id:
      user_profile = user_profile_data[user_profile_id]
      print(user_profile_id)

if user_profile == {}:
  print(f"User id {recommendation_user_id} not found. Please try with another user.")
else:
  print(user_profile.keys())


62853
dict_keys(['title', 'series', 'author', 'genres', 'pages', 'publisher', 'firstPublishDate', 'awards', 'setting', 'words'])


In [229]:
import datetime
input = "07/11/60"
format = '%d/%m/%y'
date = datetime.datetime.strptime(input, format)
if date.year > 2020:
   date = date.replace(year=date.year-100)
date

datetime.datetime(1960, 11, 7, 0, 0)

In [324]:
%%file Recomendation.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import json

#ID od the user we want recommendation for
##########################################
recommendation_user_id = "175208"
##########################################

# List of attributes that we take into account 
string_vector_attributes = ['series', 'author', 'publisher']
list_vector_attributes = ['genres', 'awards', 'setting', 'words']
int_vector_attributes = {'pages': 3000}#, 'firstPublishDate': 1000} #attribute and inverse coefficient

#Takes an input string and returns whether it's "empty"
def checkNotEmptyInput(input):
  # Check if input is not a NaN and not empty
  return input == input and input and input != " "

# Finds the user we want to give recommendation to
user_profile = {}
user_suqared_norm = 0

file_name = "user_profiles.txt"
with open(file_name, 'r', encoding='utf-8') as file:
  while (line := file.readline().rstrip()):
    user_profile_data = json.loads(line)
    user_profile_id = list(user_profile_data.keys())[0]
    if user_profile_id == recommendation_user_id:
      # Store the profile vetor
      user_profile = user_profile_data[user_profile_id]
      # Compute the norm of the user vector
      for attribute in string_vector_attributes + list_vector_attributes:
        for key, value in user_profile[attribute].items():
          if checkNotEmptyInput(key) and checkNotEmptyInput(value):
            user_suqared_norm += value**2
      for attribute, inv_coeff in int_vector_attributes.items():
          value = 0
          # Check if there is a value and this value is not a NaN
          if checkNotEmptyInput(user_profile[attribute]):
            try:
              value = (float(user_profile[attribute]) / inv_coeff)
              user_suqared_norm += (value**2)
            except ValueError:
              value = float('nan')
          user_profile[attribute] = value
      break  
      
if user_profile == {}:
  print(f"User id {recommendation_user_id} not found. Please try with another user.")
else:


  class Recomendation(MRJob):    

      def steps(self):
          #need to define the order of mappers and reducers, so that the functions takes in the right key and value
          #only one mapper and reducer per step, so therefore need several steps
          return [ 
              MRStep(mapper=self.mapper_1, reducer=self.reducer_1)
          ]

      # the map step: each book vector is read 
      def mapper_1(self, _, line):
          book = json.loads(line)
          book_id = list(book.keys())[0]
          book = book[book_id]

          #Compute the norm of the book id vector
          suqared_norm = 0
          for attribute in string_vector_attributes:
              # Check if not an empty string or NaN
              if checkNotEmptyInput(book[attribute]):
                  suqared_norm += 1
          for attribute in list_vector_attributes:
              # Check if not an empty list or NaN
              if checkNotEmptyInput(book[attribute]):
                for elmt in book[attribute]:
                  if checkNotEmptyInput(elmt):
                    suqared_norm += 1
          for attribute, inv_coeff in int_vector_attributes.items():
              value = 0
              # Check if there is a value and this value is not a NaN
              if checkNotEmptyInput(book[attribute]):# != "" or book[attribute] != " " or float(book[attribute]) != float(book[attribute]):
                  try:
                      value = (float(book[attribute]) / inv_coeff)
                      suqared_norm += value**2
                  except ValueError:
                      value = float('nan')
              book[attribute] = value
          yield book_id, (suqared_norm, book)
      
      # the reduce step: coompute the cosine similarity between the user profile and every item profile
      # Only recommend unread books
      def reducer_1(self, book_id, value):
          value = list(value)[0]
          book_suqared_norm = list(value)[0]
          book = list(value)[1]
          
          # Check that the user havez not read that book
          if book["title"] not in user_profile["title"]:

            dot_product = 0
            for attribute in string_vector_attributes:
              # Check if the value of the item vector and user vector is not an empty
              if checkNotEmptyInput(book[attribute]) and checkNotEmptyInput(user_profile[attribute]):
                for key, value in user_profile[attribute].items():
                  if key == book[attribute] and checkNotEmptyInput(value):
                    dot_product += value
            for attribute in list_vector_attributes:
              # Check if the value of the item vector and user vector is not an empty
              if checkNotEmptyInput(book[attribute]) and checkNotEmptyInput(user_profile[attribute]):
                for key, value in user_profile[attribute].items():
                  if checkNotEmptyInput(key) and checkNotEmptyInput(value) and key in book[attribute]:
                    dot_product += value
            for attribute, inv_coeff in int_vector_attributes.items():
              dot_product += (book[attribute] * user_profile[attribute])
            

            cosine_similartity = dot_product / (user_suqared_norm*book_suqared_norm) if user_suqared_norm*book_suqared_norm != 0 else 0
            yield (book_id), (cosine_similartity)

  if __name__ == '__main__':
      Recomendation.run()

Overwriting Recomendation.py


In [325]:
! python Recomendation.py item_profiles.txt > output

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\Amine\AppData\Local\Temp\Recomendation.Amine.20221129.161424.425186
Running step 1 of 1...
job output is in C:\Users\Amine\AppData\Local\Temp\Recomendation.Amine.20221129.161424.425186\output
Streaming final output from C:\Users\Amine\AppData\Local\Temp\Recomendation.Amine.20221129.161424.425186\output...
Removing temp directory C:\Users\Amine\AppData\Local\Temp\Recomendation.Amine.20221129.161424.425186...


In [341]:
import pandas as pd
books_metada = pd.read_csv("data/books_metadata.csv")
books_metada.head(1)


,book_id,title,series,author,description,genres,pages,publisher,firstPublishDate,awards,setting,coverImg
0,14796360,The Hunger Games,The Hunger Games #1,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",374,Scholastic Press,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...


In [364]:
import pandas as pd

book_similarity = []

file_name = "output"

with open(file_name, 'r', encoding='utf-8') as file:
  while (line := file.readline().rstrip()):
    line = line.split('\t')
    book_id = int(line[0].replace("\"", ""))
    similarity = float(line[1])
    book_similarity.append([book_id, similarity])

books_metada = pd.read_csv("data/books_metadata.csv")
book_similarity_df = pd.DataFrame(book_similarity, columns=["book_id", "similarity"])
book_similarity_df = book_similarity_df.merge(books_metada, on = "book_id")
book_similarity_df = book_similarity_df.sort_values(by="similarity", ascending=False)

top_ten_recommendation = book_similarity_df[["title", "author", "publisher", "coverImg"]].head(10)

# top_ten_recommendation = sorted(book_similarity, key=lambda value: value[1], reverse=True)[:10]

# pd.read_csv("data/book_metada.csv")

# Importing the necessary librariesimport pandas as pd
from IPython.core.display import HTML

# Converting links to html tags
def path_to_image_html(path):
  return '<img src="'+ path + '" width="160" >'

# Rendering the images in the dataframe using the HTML method.
HTML(top_ten_recommendation.to_html(escape=False,formatters=dict(coverImg=path_to_image_html)))

,title,author,publisher,coverImg
837,The Dark Tower Omnibus,"Robin Furth (Adaptor), Peter David (Adaptor), Jae Lee (Illustrator), Richard Isanove (Illustrator), Stephen King (Goodreads Author) (Story)",Marvel,
22433,The Day I Swapped My Dad for Two Goldfish,"Neil Gaiman (Goodreads Author), Dave McKean (Illustrator)",HarperCollins,
23609,Tintin: Complete Companion,"Michael Farr, Hergé (Illustrator)",Last Gasp,
2340,The Enchanted Castle,"E. Nesbit, H.R. Millar (Illustrator)",Puffin,
18588,The Lord of the Rings: The Making of the Movie Trilogy,"Brian Sibley, Ian McKellen (Foreword)",Mariner Books,
7760,The Rainmaker,John Grisham (Goodreads Author),Delta,
3860,Orphans of the Sky,Robert A. Heinlein,Baen,
3536,"Harry Potter and the Order of the Phoenix (Harry Potter, #5, Part 1)",J.K. Rowling,BBC Audiobooks,
7186,Star Wars: The Ultimate Visual Guide,"Daniel Wallace (Goodreads Author), Ryder Windham (Goodreads Author)",DK Children,
6426,The Tragical Comedy or Comical Tragedy of Mr. Punch,"Neil Gaiman (Goodreads Author), Dave McKean (Illustrator)",Vertigo / DC Comics,


In [347]:
import json
line = '{"14796360": {"title": "The Hunger Games", "series": "The Hunger Games #1", "author": "Suzanne Collins", "description": "WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister\'s place in the Games. But Katniss has been close to dead before\u2014and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.", "genres": ["Young Adult", "Fiction", "Dystopia", "Fantasy", "Science Fiction", "Romance", "Adventure", "Teen", "Post Apocalyptic", "Action"], "pages": "374", "publisher": "Scholastic Press", "firstPublishDate": NaN, "awards": ["Locus Award Nominee for Best Young Adult Book (2009)", "Georgia Peach Book Award (2009)", "Buxtehuder Bulle (2009)", "Golden Duck Award for Young Adult (Hal Clement Award) (2009)", "Grand Prix de l\'Imaginaire Nominee for Roman jeunesse \u00e9tranger (2010)", "Books I Loved Best Yearly (BILBY) Awards for Older Readers (2012)", "West Australian Young Readers\' Book Award (WAYRBA) for Older Readers (2010)", "Red House Children\'s Book Award for Older Readers & Overall (2010)", "South Carolina Book Award for Junior and Young Adult Book (2011)", "Charlotte Award (2010)", "Colorado Blue Spruce Young Adult Book Award (2010)", "Teen Buckeye Book Award (2009)", "Pennsylvania Young Readers\' Choice Award for Young Adults (2010)", "Rhode Island Teen Book Award (2010)", "Dorothy Canfield Fisher Children\'s Book Award (2010)", "Evergreen Teen Book Award (2011)", "Soaring Eagle Book Award (2009)", "Milwaukee County Teen Book Award Nominee (2010)", "Sakura Medal for Middle School Book (2010)", "Michigan Library Association Thumbs Up! Award (2009)", "Florida Teens Read (2009)", "Deutscher Jugendliteraturpreis for Preis der Jugendjury (2010)", "Iowa High School Book Award (2011)", "New Mexico Land of Enchantment Award for Young Adult (2011)", "Eliot Rosewater Indiana High School Book Award (2010)", "The Inky Awards for Silver Inky (2009)", "California Young Readers Medal for Young Adult (2011)", "Lincoln Award (2011)", "Kinderboekwinkelprijs (2010)", "Missouri Truman Readers Award (2011)", "CYBILS Award for Young Adult Fantasy & Science Fiction (2008)", "Literaturpreis der Jury der jungen Leser for Jugendbuch (2010)", "The Inky Awards Shortlist for Silver Inky (2009)", "Prix Et-lisez-moi (2011)", "Missouri Gateway Readers Award (2011)", "Oklahoma Sequoyah Award for High School and Intermediate (2011)", "Premio El Templo de las Mil Puertas for Mejor novela extranjera perteneciente a saga (2009)", "Rebecca Caudill Young Readers\' Book Award (2011)", "LovelyBooks Leserpreis for Fantasy (2009)", "LovelyBooks Leserpreis for Bestes Cover/Umschlag (2009)", "Premi Protagonista Jove for Categoria 13-14 anys (2010)"], "setting": ["District 12, Panem", "Capitol, Panem", "Panem (United States)"], "coverImg": "https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975l/2767052.jpg"}}'
data = json.loads(line)
book_id = list(data.keys())[0]
book = data[book_id]
book_vector_attributes = ['title', 'author', 'genres', 'pages', 'publisher', 'firstPublishDate', 'awards', 'setting']
book_vector = [book[attribute] for attribute in book_vector_attributes]
print(book['pages'])
print(data[book_id])

374
{'title': 'The Hunger Games', 'series': 'The Hunger Games #1', 'author': 'Suzanne Collins', 'description': "WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.", 'genres': ['Young Adult', 'Fiction', 'Dystop

In [6]:
user_profile = "{\"14796360\": {\"title\": \"The Hunger Games\", \"series\": \"The Hunger Games #1\", \"author\": \"Suzanne Collins\", \"description\": \"Desc.\", \"genres\": [\"Young Adult\", \"Fiction\"], \"pages\": \"374\", \"publisher\": \"Scholastic Press\", \"firstPublishDate\": \"nan\", \"awards\": [\"Locus Award Nominee for Best Young Adult Book (2009)\", \"Georgia Peach Book Award (2009)\"], \"setting\": [\"Capitol, Panem\", \"Panem (United States)\"], \"coverImg\": \"https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975l/2767052.jpg\"}}"
user_profile = json.loads(user_profile)

In [7]:
user_profile = '{"42": {"authors": [["Suzanne Collins", 2.5]], "description": "WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister\'s place in the Games. But Katniss has been close to dead before\u2014and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.", "genres": [["Young Adult", 4.5], ["Fiction",4], ["Dystopia",0], ["Fantasy",1], ["Science Fiction", 2.42], ["MegaAdventure",2.1]], "pages": "351", "publisher": [["Random publisher3", 2.5], ["Random publisher1", 3.1], ["Random publisher1", 3.1], ["Scholastic Press", 3.2]], "firstPublishDate": 1984, "awards": [["Locus Award Nominee for Best Young Adult Book (2009)", 4.1], ["Random award (2010)", 2], ["Buxtehuder Bulle (2009), NaN"], ["Golden Duck Award for Young Adult (Hal Clement Award) (2009)", 5], ["Pennsylvania Young Readers\' Choice Award for Young Adults (2010)", 2], ["Great random price (2012)", 4.2], ["Great random price (2012)", 4.2], ["Great random price (2012)", 4.2], ["Great random price (2012)", 4.2]], "setting": [["Capitol, Panem", 3.1], ["Random place", 2.5], ["Spooky place", 1.3], ["Great place", 4.1], ["Great place", 4.1], ["Wow place", 4.1], ["Surprising place", 4.1], ["Good place", 4.1]], "coverImg": "https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975l/2767052.jpg"}}'
user_profile = json.loads(user_profile)
user_profile["42"]

{'authors': [['Suzanne Collins', 2.5]],
 'description': "WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.",
 'genres': [['Young Adult', 4.5],
  ['Fiction', 4],
  ['Dystopia', 0],
  ['Fantasy', 1],
  ['Scienc

In [8]:
! python Recomendation.py test.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Traceback (most recent call last):
  File "c:\Users\Amine\Documents\Amine\INSA\INSA_5IF_DTU\Computational_Tools_For_Data_Science\Project\DataScienceFinalProject\Recomendation.py", line 62, in <module>
    Recomendation.run()
  File "C:\Users\Amine\AppData\Roaming\Python\Python39\site-packages\mrjob\job.py", line 616, in run
    cls().execute()
  File "C:\Users\Amine\AppData\Roaming\Python\Python39\site-packages\mrjob\job.py", line 687, in execute
    self.run_job()
  File "C:\Users\Amine\AppData\Roaming\Python\Python39\site-packages\mrjob\job.py", line 636, in run_job
    runner.run()
  File "C:\Users\Amine\AppData\Roaming\Python\Python39\site-packages\mrjob\runner.py", line 500, in run
    self._check_input_paths()
  File "C:\Users\Amine\AppData\Roaming\Python\Python39\site-packages\mrjob\runner.py", line 1133, in _check_input_paths
    self._check_input_path(path)
  File "C:\Users\Amine\AppDat